<h1> Introdução </h1>

<p> O tema selecionado para análise foi a <b>Avaliação de Risco de Crédito</b> por sua relevância no mercado econômico e financeiro bem como por sua complexidade, fomentando a aplicação comparativa de diversas técnicas de análises preditivas </p>

<h3> Fonte de Dados </h3>
<p> A base de dados utilizada neste estudo foi obtida no site de competições em ciência de dados kaggle.com, <a href=https://www.kaggle.com/wendykan/lending-club-loan-data/home>Lending Club Load Data </a> e consiste em um conjunto de dados completo de empréstimos concedidos a clientes entre 2007-2015, incluindo o status atual do empréstimo (calculado até à data de publicação do dataset). 
<br>Os recursos adicionais incluem pontuação de crédito, número de consultas financeiras, endereço entre outros. O arquivo é uma tabela de cerca de 890 mil observações e 75 variáveis. Um dicionário de dados (table metadata) é fornecido em um arquivo separado </p>

<h3>Objetivo</h3>
<p>Contruir um modelo que preveja se um pedido de empréstimo será honrado (loan_status="Fully Paid" ou "Current") ou não (loan_status="Demais valores")</p>

<h3>Metodologia</h3>
<p>A metodologia de análise será composta das seguintes etapas</p>
<ol>
    <li>Avaliação Preliminar</li>
    <li>Análise Descritiva</li>
    <li>Limpeza e Tratamento dos Dados</li>
    <li>Treinamento do modelo: <b>Regressão Logística</b></li>
    <li>Treinamento do modelo: <b>Árvore de Decisão</b></li>
    <li>Comparação de Performance</li>
    <li>Conclusão</li>
</ol>



<h2>1 - Avaliação Preliminar</h2>
<ul>
    <li>Leitura da base de dados</li>
    <li>Validação do tamanho da base</li>
    <li>Compreensão das variáveis disponíveis</li>
    <li>Quantificação dos valores faltantes (missing values)</li>
    <li>Identificação da variávei alvo <b>(target)</b></li>
</ul>


<h3>Leitura da base de dados</h3>

In [ ]:
#importing python data analysis libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import matplotlib.font_manager as fm

#configuring appearence styles
sns.set_style('whitegrid')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=Warning)
%matplotlib inline

fontsize2use = 12   
fontprop = fm.FontProperties(size=fontsize2use)

# Disable display truncation 
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 500)

In [ ]:
raw_data = pd.read_csv('../loan.csv',low_memory=False)

<h4>Visualização do tamanho e dimensões da base </h4>

In [ ]:
def print_shape(dataframe):
    shape = dataframe.shape
    print("As dimensões da base são: \n\n \
            Quantidade de Observações: {} \n \
            Quantidade de Variáveis  : {}".format(shape[0],shape[1]))
    
print_shape(raw_data)

<h4> Retira uma amostra aleatória do Dataset para reduzir consumo de memória </h4>
<p>O tamanho da amostra corresponde a 25% do tamanho original do dataset</p>

In [ ]:
df = raw_data.sample(frac=.25).copy()
print_shape(df)

<h4>Visualização dos nomes e metadados das colunas para compreensão das variáveis disponíveis</h4> 
<p>Neste ponto, o arquivo de metadados disponibilizado pelo kaggle é utilizado para compreender o significado de cada variável existente no dataset </p>

In [ ]:
df.columns

In [ ]:
df_metadata = pd.read_excel(io='../LCDataDictionary.xlsx').dropna()
df_metadata.style.set_properties(subset=['Description'], **{'width': '1000px'})

<h4> Quantificação dos valores faltantes (missing)</h4>

In [ ]:
#Definição de função de avaliação quantitativa de valores ausentes em um dado Pandas.DataFrame
def missing_values(dataframe):
        mis_val = dataframe.isnull().sum()
        mis_val_percent = 100 * (dataframe.isnull().sum() / len(dataframe))
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
            columns = {0 : 'Missing Values', 1 : '% of Total Values'}
        )
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0
        ].sort_values(
            '% of Total Values', ascending=False
        ).round(1)
        
        qtd_columns_over_limit = len(mis_val_table_ren_columns[mis_val_table_ren_columns['% of Total Values']>70.0])
        
        print("O Dataset possui " + str(df.shape[1]) + " colunas.\n"
                + str(mis_val_table_ren_columns.shape[0]) + 
                " Colunas possuem valores ausentes. \n" 
                + str(qtd_columns_over_limit) +
                " Colunas cuja proporção de valores ausentes ultrapassa 70% das observações")
        
        return mis_val_table_ren_columns
    
#Quantifica os valores faltantes no dataset estudado    
missing_statistics = missing_values(df)
missing_statistics[missing_statistics['% of Total Values']>70.0]

In [ ]:
plt.Figure(figsize=(15,5))
missing_statistics[
                   missing_statistics['% of Total Values'] > 0.0
                  ]['% of Total Values'].plot(
                        kind='bar',
                        figsize=(15,5),
                        fontsize=15
                    )
plt.show()

<p> O gráfico acima mostra que a base de dados possui algumas colunas com um percentual muito alto de observações nulas ou faltantes (acima de 75%, exemplos: `dti_joint`,`annual_inc_joint`,`verification_status_joint`) e que deverão ser tratadas/removidas para a modelagem</p>

In [ ]:
#exclusão de colunas com mais de 70% de valores ausentes (missing)
df.drop(labels=missing_statistics[missing_statistics['% of Total Values'] > 70.0].index,
        axis=1,
        inplace=True)
print_shape(df)

<h4> Identificação da variávei alvo (target)</h4>
<p>A coluna que contém a variável alvo do estudo é a <b>Loan Status</b>. Esta variável representa a situação corrente do título de empréstimo no momento da publicação do dataset (Current, Late, Fully Paid, etc...).</p>

In [ ]:
plt.figure(figsize=(12,6))
plt.ylabel('Loan Status')
plt.xlabel('count')
df['loan_status'].value_counts().plot(kind='barh',grid=True,fontsize=12)
plt.show()

In [ ]:
#Calcula proporção de empréstimos "Bons" e "Maus"
good_loan = len(df[(df.loan_status=='Fully Paid') |
                   (df.loan_status=='Current')])

good_loan_ratio = good_loan/len(df)
bad_loan_ratio = 1-good_loan_ratio

print("Taxa de Bons Empréstimos: %.2f%%" % (good_loan_ratio*100))
print("Taxa de Maus Empréstimos: %.2f%%" % (bad_loan_ratio*100))

pie_target_ratio = pd.Series({
    "Bons Empréstimos":(good_loan_ratio*100),
    "Maus Empréstimos":(bad_loan_ratio*100)
})

plt.figure(figsize=(5,5))
plt.pie(x=pie_target_ratio,labels=pie_target_ratio.index)
plt.show()

<h4>Observação</h4>
<p>A proporção acima mostra claramente que se trata de um caso em que a classe alvo está desbalanceada. Isto significa que uma determinada classe de observações da variável alvo (loan_status) apresenta uma proporção muito superior superior do que as demais na base e acaba dominando o modelo.</p>
<p> Esta característica deverá ser tratada ao longo do estudo, utilizando técnicas apropriadas </p>

In [ ]:
#Define a variável target (alvo)
## 1 = Empréstimo Honrado ("Fully Paid" ou "Current")
## 0 = Empréstimo Não Honrado (Demais Categorias)
df['target'] = df['loan_status'].apply(lambda value: 1 if value in ("Fully Paid","Current") else 0)
df.drop('loan_status',axis=1,inplace=True)
df.head()

<h2>2 - Análise Descritiva</h2>
<ul>
    <li>Medidas de Centralidade/Variabilidade (variáveis numéricas)</li>
    <li>Medidas de Frequência/Classes Distintas (variáveis categóricas)</li>
</ul>


<h4>Avaliação dos tipos de variáveis (Numéricas/Categóricas)</h4>
<p>Por definição (pandas.DataFrame), as variáveis numéricas no Dataset são classificadas como `float64` ou `int64`, enquanto as categóricas são classificadas como `object`</p>

In [ ]:
# Quantidade de tipos de dados diferentes no dataset
print_shape(df)
print("\n")


#função para exibir graficamente a composição de dados de um dado dataset
def print_dtypes(dataframe):
    print("Quantidade de cada Tipo de Dados existente na base \n")
    print(dataframe.dtypes.value_counts())

    plt.figure()
    dataframe.dtypes.value_counts().plot(kind='pie')
    plt.title('Número de colunas distribuído por tipo de dados',fontsize=20)
    
print_dtypes(df)

<p>O gráfico acima nos mostra que temos uma quantidade considerável de colunas com valores do tipo `object` (variáveis categóricas) que deverão ser tratadas e/ou removidas para uso no modelo </p>

<h4>Avaliação das variáveis categóricas</h4>
<p>Quantificação e frequência das classes únicas existentes em cada variável categórica do dataset</p>

In [ ]:
category = pd.DataFrame({ 'qtd_unicos': df.select_dtypes(include=['object']).apply(pd.Series.nunique, axis = 0)})  
category['perc_unicos'] = category['qtd_unicos'].apply(lambda x: (float(x)/len(df))*100)

print("Variáveis categóricas e ocorrências distintas pela base \n")
print(category.sort_values(by='perc_unicos',ascending=False))

#plota o gráfico de barras por percentual de categorias únicas
category['perc_unicos'].sort_values(ascending=False).head(10).plot.bar(
    figsize=(20,5),
    fontsize=14,
    title='Top 10 variáveis com maior % de categorias distintas')

<p> O gráfico acima nos mostra que algumas variáveis possuem alta variabilidade (número extremamente alto de categorias distintas). É o caso das variáveis `url`,`emp_title`,`title` e `title`. Estas variáveis possuem alto número de categorias distintas, o que indica aleatoriedade e prejudica a assertividade do modelo.<p>

<p> Estas variáveis serão desconsideradas na análise </p>

In [ ]:
df.drop(['url','emp_title','title'],axis=1,inplace=True)
print_shape(df)
print("\n")
print_dtypes(df)

In [ ]:
#função para plotar uma lista de gráficos de pizza com a composição de frequencias para cada variávei categórica, 
## dado um dataframe
def plot_categories_distribution(dataframe):
    categories = dataframe.select_dtypes(include=['object']).columns.sort_values()
    m = len(categories)
    fig,axes = plt.subplots(6,int(m/5),sharey=True,figsize=(10,15))

    
    for n in range(len(axes.flat)):
        try:
            ax = axes.flat[n]
            item = categories[n]
            group = dataframe.groupby(item).size()

            ax.pie(group)
            ax.set_title(item)    
        except:
            #caso não exista mais categorias a iterar, finaliza a execução da função
            return

In [ ]:
df.select_dtypes(include=['object']).describe()

<p> A tabela acima mostra de forma resumida a composição de cada variável categórica existente. Estão consolidadas medidas básicas como 
<ul> 
    <li>Quantidade (count) </li> 
    <li> Quantidade de Valores distintos (unique)</li> 
    <li>Classe mais comum (top)</li>
    <li>Frequência da classe mais comum (freq)</li>
</ul>
</p>
<p> É possível verificar que muitas variáveis são compostas de texto puro/alta variação de categorias, que deverão ser codificados e/ou removidos para treinamento do modelo </p>
<p> Além disso, algumas variáveis apresentam variação quase nula na base (ex.: `pymnt_plan` e `application_type`), o que indica que tais variáveis deverão ser removidas </p>

In [ ]:
plot_categories_distribution(df)

<p> A relação de gráficos acima ilustra de forma visual a frequência por categoria em cada uma das variáveis categóricas do modelo </p>

In [ ]:
# Remove as colunas pymnt_plan e application_type
df.drop([    'zip_code',
             'verification_status',
             'home_ownership',
             'issue_d',
             'earliest_cr_line',
             'last_pymnt_d',
             'next_pymnt_d',
#             'desc',
#             'pymnt_plan',
#             'initial_list_status',
#             'addr_state',
             'last_credit_pull_d'],axis=1,inplace=True)

In [ ]:
df.select_dtypes(include=['object']).head(10)

<h4>Avaliação das variáveis numéricas</h4>
<p>Medidas de Centralidade e Variabilidade existentes em cada variável numérica do dataset. </p>

In [ ]:
df.select_dtypes(include=['float64','int64']).columns

<p> Removendo as colunas `id` e `member_id` pois tratam-se de valores únicos de identificação dos contratos no dataset e não têm valor para a análise </p> 

In [ ]:
df.drop(['id','member_id'],axis=1, inplace=True)
print_shape(df)

In [ ]:
def plot_numeric_histogram(dataframe):
    categories = dataframe.select_dtypes(include=['float64','int64']).columns
    m = len(categories)
    fig,axes = plt.subplots(6,int(m/5),figsize=(30,35))

    
    for n in range(len(axes.flat)-1):
        try:
            ax = axes.flat[n]
            item = categories[n]
            group = dataframe.groupby(item).size()
        
            ax.hist(group,bins=10,)
            ax.set_title(item,fontsize=10)  
        except:
            #caso não encontre mais itens  para iterar
            return

In [ ]:
df.select_dtypes(include=['float64','int64']).describe()

<p> A tabela acima mostra de forma resumida um conjunto básico de medidas descritivas para cada variávei numérica existente no dataset. Entre elas:
<ul> 
    <li>Quantidade (count) </li> 
    <li> Média (mean)</li> 
    <li> Desvio Padrão (std)</li>
    <li> Valores Máximos e mínimos (max/min)</li>
    <li> Percentis (25%, 50%, 75%) </li>
</ul>
</p>

In [ ]:
plot_numeric_histogram(df)

<h2>3 - Limpeza e Tratamento dos Dados</h2>
<ul>
   <li>Transformação das variáveis categóricas relevantes (encoding e hot-encoding)</li>
   <li>Normalização/Agrupamento das variáveis numéricas </li>
</ul>


In [ ]:
df.info()

<p> Primeiramente, os valores de data serão convertidos em inteiros pelo número do ano para evitar crescimento muito grande de colunas pelo hot-encoding. Para preencher os valores faltantes, foi utilizada a data com o maior número de ocorrências </p>

In [ ]:
df['issue_d']= pd.to_datetime(df['issue_d']).apply(lambda x: int(x.strftime('%Y')))
df['last_pymnt_d']= pd.to_datetime(df['last_pymnt_d'].fillna('2016-01-01')).apply(lambda x: int(x.strftime('%m')))
df['last_credit_pull_d']= pd.to_datetime(df['last_credit_pull_d'].fillna("2016-01-01")).apply(lambda x: int(x.strftime('%m')))
df['earliest_cr_line']= pd.to_datetime(df['earliest_cr_line'].fillna('2001-08-01')).apply(lambda x: int(x.strftime('%m')))
df['next_pymnt_d'] = pd.to_datetime(df['next_pymnt_d'].fillna(value = '2016-02-01')).apply(lambda x:int(x.strftime("%Y")))

<h4>emp_length</h4>
<p>Os valores faltantes serão preenchidos com 0 assumindo que o tomador do empréstimo não trabalhou até a data de medição </p>

In [ ]:
df['emp_length'].fillna(value=0,inplace=True)
df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df['emp_length'].value_counts().sort_values().plot(kind='bar',figsize=(18,8))
plt.title('Number of loans distributed by Employment Years',fontsize=20)
plt.xlabel('Number of loans',fontsize=15)
plt.ylabel('Years worked',fontsize=15);

<p> Aplica o hot-encoding nas variáveis categóricas </p>

In [ ]:
from sklearn import preprocessing

def label_encode(dataframe):
    for col in dataframe:
        if dataframe[col].dtype == 'object':
            if len(list(dataframe[col].unique())) <= 2:     
                le = preprocessing.LabelEncoder()
                dataframe[col] = le.fit_transform(dataframe[col])
                count += 1
                print (col)
            
    print('%d columns were label encoded.' % count)

<p> Remove as variáveis que sofreram o hot-encoding </p>